In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

# Load file

In [8]:
PATH = r'transaction_history.csv'
df = pd.read_csv(PATH)

# Clean and prepare data 

In [9]:
# rename right value from Method column to 'Purchase of NC coins'
con_1 = (df.Method == "Swap Exact Token...") & (df.From == '0x78e16d2facb80ac536887d1376acd4eeedf2fa08')
con_2 = (df.Method == "Swap")
con_3 = (df.Method == "Swap ETH For Exa...")
con_4 = (df.Method == "Swap Exact ETH F...") 
con_5 = (df.Method == "0x415565b0") 
df.loc[con_1 | con_2 | con_3 | con_4 | con_5, 'Method'] = 'Purchase of NC coins'

# rename right value from Method column to 'Sales of NC coins'
con_1 = (df.Method == "Swap Exact Token...") & (df.From != '0x78e16d2facb80ac536887d1376acd4eeedf2fa08')
con_2 = (df.Method == "0x0773b509")
df.loc[con_1 | con_2, 'Method'] = 'Sales of NC coins'

# rename 'Add Liquidity ET...' to 'Add Liquidity'
con_1 = (df.Method == "Add Liquidity ET...")
df.loc[con_1, 'Method'] = 'Add Liquidity'

# rename 'Remove Liquidity...' to 'Remove Liquidity'
con_1 = (df.Method == "Remove Liquidity...")
df.loc[con_1, 'Method'] = 'Remove Liquidity'

# remove useless value form Method column
value_to_remove = df.loc[(df.Method == "Stake") | (df.Method == "Fund") | (df.Method == "0xf574133c") | (df.Method == "0x6d9cec22") | (df.Method == "Claim") | (df.Method == "Unstake")].index
df = df.drop(value_to_remove, axis=0)

# remove useless columns
df = df.drop(["Unnamed: 0", "Unnamed: 4", "Unnamed: 7"], axis=1)

# split 'Date Time (UTC)' into two columns 'Date' and "Time", then format to 'Date' to datetime format
df['Date'], df['Time'] = zip(*df['Date Time (UTC)'].str.split().tolist())
del df['Date Time (UTC)']
df["Date"] = pd.to_datetime(df["Date"])

In [10]:
df.head()

,Txn Hash,Method,From,To,Quantity,Date,Time
0,0x38f5b276912bacc831bd5f1ce9d415177eb217216f86...,Add Liquidity,0x98edd7629f157640b256b2035fe39f8624ebd0d8,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,254203.587367,2022-01-28,10:13:57
1,0xde2711d68d09c88801421bdb50df1486c654bd00688d...,Sales of NC coins,0x98edd7629f157640b256b2035fe39f8624ebd0d8,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,260300.000000,2022-01-28,10:11:55
3,0x1b3c6d2747c9eb2867d4f25db5a79a6d6248079a2db8...,Add Liquidity,0x98edd7629f157640b256b2035fe39f8624ebd0d8,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,207499.031873,2022-01-27,15:24:56
4,0x64fcf0b3cad709a4ac7b6b45f6ab91eb1d4f98cb2173...,Sales of NC coins,0x98edd7629f157640b256b2035fe39f8624ebd0d8,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,213750.000000,2022-01-27,15:22:30
6,0xf3d64d9d794c9184261c879a91cb881c5794fda7464b...,Purchase of NC coins,0x78e16d2facb80ac536887d1376acd4eeedf2fa08,0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3,95094.293446,2022-01-26,23:42:45


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 701 entries, 0 to 719
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Txn Hash  701 non-null    object        
 1   Method    701 non-null    object        
 2   From      701 non-null    object        
 3   To        701 non-null    object        
 4   Quantity  701 non-null    float64       
 5   Date      701 non-null    datetime64[ns]
 6   Time      701 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 43.8+ KB


In [12]:
print(f'Unique From Adress {len(df.From.unique())}')
print(f'Unique To Adress {len(df.To.unique())}')

Unique From Adress 44
Unique To Adress 92


# Who add to liquidity pool NC coin?

In [13]:
# data for table
adresses = df.loc[df.Method == "Add Liquidity", 'From'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Add Liquidity") & (df.From == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Add Liquidity") & (df.From == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number_of_transaction': num_transaction,
    'Value': value_list}

df_lp = pd.DataFrame(data=d, columns=['Adress',  'Number_of_transaction', 'Value'])

# sort by the biggest input to LP
df_lp = df_lp.sort_values(by='Value', ascending=False)

# data for plots
df_temp = df.loc[(df.Method == 'Add Liquidity')]

# aggregate by day
df_temp = df_temp.groupby(by=[df_temp.Date]).mean().reset_index()

In [17]:
# Set plot parametr

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        line = dict(color = 'blue'),
        opacity = 0.3,
        mode='lines'
    ),    
    row=3, col=1
), 

fig.add_trace(
    go.Bar(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        marker_color='blue'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
    columnorder = [1,2,3],
    columnwidth = [100,50,50],
    header=dict(
        values=[['<b>Adress</b>'],['<b>Numer Of Transaction</b>'],['<b>Value</b>']],

        align='left',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[df_lp.Adress, df_lp.Number_of_transaction, df_lp.Value],

        align=['left'],
              )),
    row=1, col=1
)
fig.update_layout(
    height=1200,
    showlegend=False,
    title_text="Contribution to the liquidity pool",
)

fig.show()

# Who buy the most NC coin?

In [18]:
# data for table
adresses = df.loc[df.Method == "Purchase of NC coins", 'To'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Purchase of NC coins") & (df.To == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Purchase of NC coins") & (df.To == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number_of_transaction': num_transaction,
    'Value': value_list}

df_buy = pd.DataFrame(data=d, columns=['Adress',  'Number_of_transaction', 'Value'])

# sort by the number of transaction
df_buy = df_buy.sort_values(by='Number_of_transaction', ascending=False)

# data for plots
df_temp = df.loc[(df.Method == 'Purchase of NC coins')]

# aggregate by day
df_temp = df_temp.groupby(by=[df_temp.Date]).mean().reset_index()

In [19]:
# Set plot parametr

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        line = dict(color = 'blue'),
        opacity = 0.3,
        mode='lines'
    ),    
    row=3, col=1
), 

fig.add_trace(
    go.Bar(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        marker_color='blue'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
    columnorder = [1,2,3],
    columnwidth = [100,50,50],
    header=dict(
        values=[['<b>Adress</b>'],['<b>Numer Of Transaction</b>'],['<b>Value</b>']],

        align='left',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[df_buy.Adress, df_buy.Number_of_transaction, df_buy.Value],

        align=['left'],
              )),
    row=1, col=1
)
fig.update_layout(
    height=1200,
    showlegend=False,
    title_text="Purchases of NC coin",
)

fig.show()

#  Who sold the most NC coin? 

In [21]:
# data for table
adresses = df.loc[df.Method == "Sales of NC coins", 'From'].value_counts().index
adresses = adresses.tolist()
value_list = []
num_transaction = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Sales of NC coins") & (df.From == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    trans_total = len(df.loc[(df.Method == "Sales of NC coins") & (df.From == adress)].value_counts())
    num_transaction.append(trans_total)
    
d = {'Adress': adresses,
    'Number_of_transaction': num_transaction,
    'Value': value_list}

df_sell = pd.DataFrame(data=d, columns=['Adress',  'Number_of_transaction', 'Value'])

# sort by the number of transaction
df_sell = df_sell.sort_values(by='Number_of_transaction', ascending=False)

# data for plots
df_temp = df.loc[(df.Method == 'Sales of NC coins')]

# aggregate by day
df_temp = df_temp.groupby(by=[df_temp.Date]).mean().reset_index()

In [25]:
# Set plot parametr

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        line = dict(color = 'blue'),
        opacity = 0.3,
        mode='lines'
    ),    
    row=3, col=1
), 

fig.add_trace(
    go.Bar(
        x=df_temp['Date'],
        y=df_temp['Quantity'],
        marker_color='blue'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
    columnorder = [1,2,3],
    columnwidth = [100,50,50],
    header=dict(
        values=[['<b>Adress</b>'],['<b>Numer Of Transaction</b>'],['<b>Value</b>']],

        align='left',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[df_sell.Adress, df_sell.Number_of_transaction, df_sell.Value],

        align=['left'],
              )),
    row=1, col=1
)
fig.update_layout(
    height=1200,
    showlegend=False,
    title_text="Sales of NC coins",
)

fig.show()

#  Who collect Airdrop?

In [29]:
# data for table
adresses = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To'].value_counts().index
adresses = adresses.tolist()
value_list = []

for adress in adresses:
    value_total = df.loc[(df.Method == "Transfer") & (df.To == adress), 'Quantity'].sum()
    value_list.append(round(value_total, 2))
    
d = {'Adress': adresses,

    'Value': value_list}

df_ad = pd.DataFrame(data=d, columns=['Adress', 'Value'])

# sort by the number of transaction
df_ad = df_ad.sort_values(by='Value', ascending=False)

In [30]:
fig = go.Figure(data=[go.Table(
    columnorder = [1,2],
    columnwidth = [100,50],
    header=dict(
        values=[['<b>Adress</b>'], ['<b>Value</b>']],
        fill_color='paleturquoise',
        align='center',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[df_ad.Adress, df_ad.Value],
        fill_color='lavender',
        align=['left','center'],
              ))
])

fig.update_layout(title='Airdop members')
fig.show()

In [31]:
x = adresses = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To']
print(f'Total number of Airdrop: {len(x)}')

Total number of Airdrop: 106


#  Who sell the  Airdrop? 

In [32]:
# data for table
adresses_airdrop = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To'].value_counts().index
adresses_airdrop = adresses_airdrop.tolist()
seller_adresses = df.loc[(df.Method == "Sales of NC coins"), 'From']
seller_adresses = seller_adresses.tolist()
airdrop_seller_list = []

for seller_adress in seller_adresses:
    if seller_adress in adresses_airdrop:
        airdrop_seller_list.append(seller_adress)

df_ad_seller = pd.DataFrame(data=airdrop_seller_list, columns=['Adress'])

In [34]:
fig = go.Figure(data=[go.Table(
    columnorder = [1],
    columnwidth = [50],
    header=dict(
        values=[['<b>Adress</b>']],
        fill_color='paleturquoise',
        align='center',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[df_ad_seller.Adress],
        fill_color='lavender',
        align=['center'],
              ))
])

fig.update_layout(title='Who selled airdop')
fig.show()

# Who didn't add NC Coin to LP

In [35]:
# Who add NC Coin to LP
who_add_to_lp = df.loc[df.Method == "Add Liquidity", 'From'].value_counts().index

# Who bought NC Coin 
who_bought_nc = df.loc[df.Method == "Purchase of NC coins", 'To'].value_counts().index

# Who recive air drop
who_recive_ad = df.loc[(df.Method == "Transfer") & (df.From == '0xba93f37118f9d0ac9f620cd7bfb9fd79635db7d3'), 'To'].value_counts().index

# Data Series to list 
who_recive_ad = who_recive_ad.str.split().tolist()
who_bought_nc = who_bought_nc.str.split().tolist()
who_add_to_lp = who_add_to_lp.str.split().tolist()

# NC Coin holders list
who_have_nc_list = []
who_have_nc_list.extend(who_recive_ad)
who_have_nc_list.extend(who_bought_nc)

# Remove duplicates from who_have_nc_list
who_have_nc_list_final = []

for i in who_have_nc_list:
    if i not in who_have_nc_list_final:
        who_have_nc_list_final.append(i)
        
# Subtract a `who_have_nc_list_final` list from a `who_add_to_lp` list
not_add_to_lp = [adress for adress in who_have_nc_list_final if adress not in who_add_to_lp]

In [36]:
print(f'How many people did not add NC coin to LP: {len(not_add_to_lp)}')

How many people did not add NC coin to LP: 55


In [37]:
fig = go.Figure(data=[go.Table(
    columnorder = [1],
    columnwidth = [50],
    header=dict(
        values=[['<b>Adress</b>']],
        fill_color='paleturquoise',
        align='center',
        font_size=12,
        line_color='darkslategray'
               ),
    cells=dict(
        values=[not_add_to_lp],
        fill_color='lavender',
        align=['center'],
              ))
])

fig.update_layout(title='Who did not add NC Coin to LP')
fig.show()